<a href="https://colab.research.google.com/github/elizabethyoo/team_9/blob/data%2FBFS_mf/Complie_sbp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import glob
import re
import numpy as no
import seaborn as sns
import matplotlib.pyplot as plt

# Get the file names in the folder

In [3]:
files = glob.glob(f"/content/drive/MyDrive/Group 9 Project Documents/Datasets/business_pulse/*.xlsx") 

In [4]:
emp_files = [x for x in files if 'empclass' in x]
st_sector_files = [x for x in files if '/national_state_sector' in x]
st_files = [x for x in files if '/national_state_' in x and not '/national_state_sector' in x]
naics3_files = [x for x in files if 'naics3' in x]
indexes_files = [x for x in files if 'indexes' in x]
msa_files = [x for x in files if 'top_50_msa' in x]
response_urr_files = [x for x in files if 'response_urr' in x]

# Explore data structure of each data set

In [5]:
# Indexes
pd.read_excel(indexes_files[0]).head(2)

,ST,NAICS_SECTOR,INDEX_NAME,ESTIMATE_PERCENTAGE
0,-,-,Expected recovery index,-0.58%
1,-,-,Financial stress index,-0.39%


In [6]:
# Exmployee size
pd.read_excel(emp_files[0]).head(2)

,ST,NAICS_SECTOR,EMPCLASS,EMPLOYEE_SIZE,INSTRUMENT_ID,QUESTION,ANSWER_ID,ANSWER_TEXT,ESTIMATE_PERCENTAGE,SE
0,-,-,-,-,1,"Overall, how has this business been affected b...",1,Large negative effect,51.4%,0.21%
1,-,-,-,-,1,"Overall, how has this business been affected b...",2,Moderate negative effect,38.5%,0.27%


In [7]:
# response_urr_files
pd.read_excel(response_urr_files[0]).head(2)

,STATE_TAB,RESPONSE_COUNT,URR
0,AK,80,22%
1,AL,250,19%


In [8]:
# st_sector_files
pd.read_excel(st_sector_files[0]).head(2)

,ST,NAICS_SECTOR,INSTRUMENT_ID,QUESTION,ANSWER_ID,ANSWER_TEXT,ESTIMATE_PERCENTAGE,SE
0,-,-,1,"Overall, how has this business been affected b...",1,Large negative effect,51.4%,0.21%
1,-,-,1,"Overall, how has this business been affected b...",2,Moderate negative effect,38.5%,0.27%


## Note:
1. Data doesn't have date/time column.
2. Percentage sign is included in the dataset.


## To do:
1. Add date to the dataframe to indicate when the data is collected. 
2. Strip the "%" out so we can treat the value as float data type.

In [9]:
indexes_files[0]

'/content/drive/MyDrive/Group 9 Project Documents/Datasets/business_pulse/indexes_26Apr20_02May20.xlsx'

In [10]:
def p2f(x):
    return float(x.strip('%'))

def add_date_period_to_df(files_name):
  # Getting file name --getting date range of the data
  
  regex = r'\d+[A-Z][a-z]+\d+_\d+[A-Z][a-z]+\d+'  
  period_str = re.search(regex, files_name.split('/')[-1])[0]
  date_period = period_str.split('_')


  date_start = pd.to_datetime(date_period[0], format= '%d%b%y')
  date_end = pd.to_datetime(date_period[1], format= '%d%b%y')
  
  # the data that was in percentage got turn into fraction
  df = pd.read_excel(files_name,  converters={'ESTIMATE_PERCENTAGE':p2f, 'SE':p2f}).assign(DATE_START=date_start).assign(DATE_END=date_end)
  return df

In [11]:
test_index_df = add_date_period_to_df(indexes_files[0])
test_index_df.head()

,ST,NAICS_SECTOR,INDEX_NAME,ESTIMATE_PERCENTAGE,DATE_START,DATE_END
0,-,-,Expected recovery index,-0.58,2020-04-26,2020-05-02
1,-,-,Financial stress index,-0.39,2020-04-26,2020-05-02
2,-,-,Operational challenges index,-0.45,2020-04-26,2020-05-02
3,-,-,Overall sentiment index,-0.69,2020-04-26,2020-05-02
4,-,11,Expected recovery index,-0.49,2020-04-26,2020-05-02


# Append date into 1 big data set by data type


## Index Files

In [12]:
# append data into 1 big date set
df_index_all = pd.DataFrame()

# loop thru all files in the list and concat them into one df
for file in indexes_files:
  df_index = add_date_period_to_df(file)
  df_index_all =pd.concat([df_index_all, df_index])

In [13]:
df_index_all.shape

(13582, 6)

In [14]:
df_index_all.head()

,ST,NAICS_SECTOR,INDEX_NAME,ESTIMATE_PERCENTAGE,DATE_START,DATE_END
0,-,-,Expected recovery index,-0.58,2020-04-26,2020-05-02
1,-,-,Financial stress index,-0.39,2020-04-26,2020-05-02
2,-,-,Operational challenges index,-0.45,2020-04-26,2020-05-02
3,-,-,Overall sentiment index,-0.69,2020-04-26,2020-05-02
4,-,11,Expected recovery index,-0.49,2020-04-26,2020-05-02


In [15]:
df_index_all.tail()

,ST,NAICS_SECTOR,INDEX_NAME,ESTIMATE_PERCENTAGE,DATE_START,DATE_END
279,WV,-,Overall sentiment index,-0.35,2021-09-27,2021-10-03
280,WY,-,Expected recovery index,-0.40,2021-09-27,2021-10-03
281,WY,-,Market challenges index,0.09,2021-09-27,2021-10-03
282,WY,-,Operational challenges index,-0.16,2021-09-27,2021-10-03
283,WY,-,Overall sentiment index,-0.25,2021-09-27,2021-10-03


In [16]:
df_index_all.dtypes

ST                             object
NAICS_SECTOR                   object
INDEX_NAME                     object
ESTIMATE_PERCENTAGE           float64
DATE_START             datetime64[ns]
DATE_END               datetime64[ns]
dtype: object

## State & NAICS

In [17]:
# append data into 1 big date set
df_st_sector_all = pd.DataFrame()

# loop thru all files in the list and concat them into one df
for file in st_sector_files:
  df_index = add_date_period_to_df(file)
  df_st_sector_all =pd.concat([df_st_sector_all, df_index])

In [18]:
df_st_sector_all.shape

(911072, 10)

In [19]:
df_st_sector_all.head()

,ST,NAICS_SECTOR,INSTRUMENT_ID,QUESTION,ANSWER_ID,ANSWER_TEXT,ESTIMATE_PERCENTAGE,SE,DATE_START,DATE_END
0,-,-,1,"Overall, how has this business been affected b...",1,Large negative effect,51.4,0.21,2020-04-26,2020-05-02
1,-,-,1,"Overall, how has this business been affected b...",2,Moderate negative effect,38.5,0.27,2020-04-26,2020-05-02
2,-,-,1,"Overall, how has this business been affected b...",3,Little or no effect,7.6,0.29,2020-04-26,2020-05-02
3,-,-,1,"Overall, how has this business been affected b...",4,Moderate positive effect,1.7,0.08,2020-04-26,2020-05-02
4,-,-,1,"Overall, how has this business been affected b...",5,Large positive effect,0.8,0.06,2020-04-26,2020-05-02


In [20]:
df_st_sector_all.tail()

,ST,NAICS_SECTOR,INSTRUMENT_ID,QUESTION,ANSWER_ID,ANSWER_TEXT,ESTIMATE_PERCENTAGE,SE,DATE_START,DATE_END
17363,WY,-,7,"In the last week, did this business have a cha...",3,No change,76.5,3.93,2021-09-27,2021-10-03
17364,WY,-,8,"In the last week, did this business require em...",2,No,78.0,7.00,2021-09-27,2021-10-03
17365,WY,-,9,"In the last week, did this business require em...",2,No,75.5,8.02,2021-09-27,2021-10-03
17366,WY,62,13,Comparing now to what was normal before March ...,3,Little or no change in number of hours worked ...,100.0,0.00,2021-09-27,2021-10-03
17367,WY,62,5,"In the last week, did this business do any of ...",4,None of the above,100.0,0.00,2021-09-27,2021-10-03


In [21]:
df_st_sector_all.dtypes

ST                             object
NAICS_SECTOR                   object
INSTRUMENT_ID                   int64
QUESTION                       object
ANSWER_ID                       int64
ANSWER_TEXT                    object
ESTIMATE_PERCENTAGE           float64
SE                            float64
DATE_START             datetime64[ns]
DATE_END               datetime64[ns]
dtype: object

# TO DO

1. The state name is in abbreviation. Data structure/ value for state name is different from the household data. Therefore, a conversion of the state name is needed. (Either on the small business dataset or the household dataset)

2. Select features that can be used as an indicator for business recovery.

3. from the household datasets, find interesting variables that might be associated with the business recovery. 

In [22]:
state_name_mapping = {
  "AL": "Alabama",
  "AK": "Alaska",
  "AZ": "Arizona",
  "AR": "Arkansas",
  "CA": "California",
  "CO": "Colorado",
  "CT": "Connecticut",
  "DE": "Delaware",
  "FL": "Florida",
  "GA": "Georgia",
  "HI": "Hawaii",
  "ID": "Idaho",
  "IL": "Illinois",
  "IN": "Indiana",
  "IA": "Iowa",
  "KS": "Kansas",
  "KY": "Kentucky",
  "LA": "Louisiana",
  "ME": "Maine",
  "MD": "Maryland",
  "MA": "Massachusetts",
  "MI": "Michigan",
  "MN": "Minnesota",
  "MS": "Mississippi",
  "MO": "Missouri",
  "MT": "Montana",
  "NE": "Nebraska",
  "NV": "Nevada",
  "NH": "New Hampshire",
  "NJ": "New Jersey",
  "NM": "New Mexico",
  "NY": "New York",
  "NC": "North Carolina",
  "ND": "North Dakota",
  "OH": "Ohio",
  "OK": "Oklahoma",
  "OR": "Oregon",
  "PA": "Pennsylvania",
  "RI": "Rhode Island",
  "SC": "South Carolina",
  "SD": "South Dakota",
  "TN": "Tennessee",
  "TX": "Texas",
  "UT": "Utah",
  "VT": "Vermont",
  "VA": "Virginia",
  "WA": "Washington",
  "WV": "West Virginia",
  "WI": "Wisconsin",
  "WY": "Wyoming"
}